# Vitessce Widget Tutorial

# Visualization of a Loom file

## 1. Import dependencies

In [ ]:
import os
from os.path import join
from urllib.request import urlretrieve
from anndata import read_loom

from vitessce import (
    VitessceConfig,
    Component as cm,
    CoordinationType as ct,
    AnnDataWrapper,
)

## 2. Download data

Download `osmFISH_SScortex_mouse_all_cells.loom` from http://loom.linnarssonlab.org/.

In [ ]:
os.makedirs("data", exist_ok=True)
loom_filepath = join("data", "osmFISH_SScortex_mouse_all_cells.loom")
urlretrieve('http://loom.linnarssonlab.org/clone/osmFISH/osmFISH_SScortex_mouse_all_cells.loom', loom_filepath)

## 3. Open Loom file with AnnData's read_loom

In [ ]:
adata = read_loom(loom_filepath, obsm_names={"tSNE": ["_tSNE_1", "_tSNE_2"], "spatial": ["X", "Y"]})

## 4. Configure Vitessce

Create a Vitessce view config.

In [ ]:
vc = VitessceConfig(name='Loom Example', description='osmFISH dataset of the mouse cortex including all cells')
w = AnnDataWrapper(adata, cell_set_obs=["ClusterName"], cell_set_obs_names=["Clusters"], spatial_centroid_obsm="spatial", mappings_obsm=["tSNE"])
dataset = vc.add_dataset(name='SScortex').add_object(w)

tsne = vc.add_view(cm.SCATTERPLOT, dataset=dataset, mapping="tSNE")
cell_sets = vc.add_view(cm.CELL_SETS, dataset=dataset)
spatial = vc.add_view(cm.SPATIAL, dataset=dataset)

vc.link_views([spatial], [ct.SPATIAL_ZOOM, ct.SPATIAL_TARGET_X, ct.SPATIAL_TARGET_Y], [-6.43, 10417.69, 24885.55])
vc.layout(spatial | (tsne / cell_sets));

## 5. Render the widget

A widget can be created with the `.widget()` method on the config instance. Here, the `proxy=True` parameter allows this widget to be used in a cloud notebook environment, such as Binder.

In [ ]:
vw = vc.widget()
vw